In [1]:
def hour_to_date(dt):
    return dt.date()

def round_to_hour(dt):
    dt_start_of_hour = dt.replace(minute=0, second=0, microsecond=0)
    return dt_start_of_hour

def SetColor(x,mean_value):
    if(x ==0):
        return "grey"
    else:
        if x > mean_value:
            return "red"
        if x <=mean_value:
            return "green"
        #         return "black"
def SetOpacity(x):
    if(x ==0):
        return 0
    else:
        return 1

def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    from itertools import repeat
    import random,pickle
#     import plotly
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    with open('../pickled_files/user_days_week.pickle', 'rb') as handle:
        user_days_week = pickle.load(handle)
    with open('../pickled_files/not_stressed_df.pickle', 'rb') as handle:
        final_df_selected_not_stressed = pickle.load(handle)
    non_stressed_df=final_df_selected_not_stressed.loc[final_df_selected_not_stressed["user"]==user_id]
        
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    user_df["start_hour"]=user_df["starttime"].apply(round_to_hour)
    user_df["end_hour"]=user_df["endtime"].apply(round_to_hour)
    non_stressed_df["start_hour"]=non_stressed_df["starttime"].apply(round_to_hour)
    non_stressed_df["end_hour"]=non_stressed_df["endtime"].apply(round_to_hour)
    
    
    if len(user_df)>0:
        user_df["counter1"]=user_df["counter"]
        non_stressed_df["counter1"]=non_stressed_df["counter"]
        user_df_group=user_df.groupby(["counter1"],as_index=False)["counter"].count()
        not_stressed_df_group=non_stressed_df.groupby(["counter1"],as_index=False)["counter"].count()
        counter_dict_mod={}
        counter_dict=dict(zip(user_df_group.counter1, user_df_group.counter))
        counter_dict1=dict(zip(not_stressed_df_group.counter1, not_stressed_df_group.counter))
        for counter in range(0,max(user_df_group["counter1"])+1):
            if counter in counter_dict:
                counter_dict_mod[counter]=counter_dict[counter]
            else:
                if counter in counter_dict1:
                    counter_dict_mod[counter]=0.01
                else:
                    counter_dict_mod[counter]=0
        mean_value=sum(counter_dict.values()) / len(counter_dict)
        max_value=max(counter_dict.values())
        max_counter=max(counter_dict.keys())
        y=list(counter_dict_mod.values())
        x=list(counter_dict_mod.keys())
        count_day_user=user_days_week[user_id][0]["Count_days"]
        week_day_user=user_days_week[user_id][0]["Weeks_days"]
        counter_day_dict={}
        days=[]
        for count in range(1,max_counter+1):
            counter_day_dict[count]=list(count_day_user.keys())[list(count_day_user.values()).index(count)]
            days.append(str(list(count_day_user.keys())[list(count_day_user.values()).index(count)]))

        week_vals=[]
        for week in range(1,list(week_day_user.items())[-1][1]+1):
            week_vals.append(list(week_day_user.keys())[list(week_day_user.values()).index(week)])
        
        fig = go.Figure(data=go.Scatter( x=days,y=y, mode='markers', marker=dict(size=8,color= [SetColor(p, mean_value) for p in y],opacity=list(map(SetOpacity, y))),line={"dash":"dash","width":0.5}))

        # fig.add_hrect(y0=mean_value,  y1=max_value, line_width=0, fillcolor="red", opacity=0.5)
        # fig.add_hrect( y0=mean_value,  y1=0, line_width=0, fillcolor="green", opacity=0.5)
        fig.add_hline( y=mean_value,fillcolor="black", opacity=0.5)
        # fig.add_hline( y=median,fillcolor="blue", opacity=0.5)
        # fig.add_hline( y=Q1,fillcolor="yellow", opacity=0.5)
        # fig.add_hline( y=Q3,fillcolor="cyan", opacity=0.5)
        fig.update_yaxes(dtick=1)
        fig.update_xaxes( zeroline=False,range=[0,max_counter+1],type="category",tickvals=week_vals,tickangle=90)
        fig.update_yaxes(zeroline=False,range=[0,max_value+0.5],)
        fig.update_layout(template="plotly_white")
        fig.update_yaxes(showgrid=True ,
           
            showline=True,linewidth=1,
            
        )
        fig.update_xaxes(side="bottom",
                  showline=True,linecolor="Black",
            )
        fig.update_layout(template="plotly_white")
#         fig.layout.xaxis.color = 'Black'
#         fig.layout.yaxis.color = 'gray'
#         fig.update_xaxes(fixedrange=True)
#         fig.update_yaxes(fixedrange=True)
#         fig.layout.xaxis.gridcolor="black"
#         fig.layout.xaxis.gridwidth=0.5
#         fig.layout.xaxis.tickfont={"family":"Times New Roman","size":15}
#         fig.layout.yaxis.gridcolor="black"
#         fig.layout.yaxis.gridwidth=0.5
        data_summary_user=data_summary_user_current["Total_Trend"]
      

       
       
        
        
        return "Total_Trend",fig,data_summary_user
    else:
        return "","",""